# Machine Learning

__Machine learning__ is a method of _data analysis_ that automates
_analytical model building_.
It is a branch of _artificial intelligence_ based on the idea that systems
can _learn from data_, _identify patterns_ and _make decisions_ with
minimal human intervention.

## Sci-kit Learn (SKlearn, Scipy, Numpy)

__Scikit-learn__ is a _Python package_ that provides a wide range of _machine learning algorithms_ and tools. 
It is built on top of _NumPy_, _SciPy_, and _Matplotlib_, and is designed to be simple and efficient for data analysis and modeling.

__Scikit-learn__ offers various modules for tasks such as _classification_, _regression_, _clustering_, _dimensionality reduction_, and _model selection_.
It also provides utilities for _preprocessing data_, _evaluating models_, and _handling datasets_.

With its extensive documentation and user-friendly interface, __Scikit-learn__ is widely used in the field of machine learning and data science.

In [1]:
#!pip install scikit-learn
import numpy as np
import pandas as pd
import sklearn

### K-Nearest Neighbors

__K-Nearest Neighbors__ is a simple algorithm that _stores all available
cases_ and _classifies_ new cases based on a similarity measure.

It is a type of _instance-based learning_, or _lazy learning_, where the
function is only approximated locally and all computation is deferred
until function evaluation.

In [2]:
# Classification of the data
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

titanic_df = pd.read_csv('datasets/titanic.csv')
titanic_df = titanic_df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Survived']]
titanic_df = titanic_df.dropna().reset_index(drop=True)

X = titanic_df.drop(['Survived'], axis=1)  
y = titanic_df['Survived']

# encode sex column using OneHotEncoder
encoder = OneHotEncoder()
encoded_data = encoder.fit_transform(X[['Sex']]) # Encoding 'Sex' column
encoded_data_dense = encoded_data.toarray()
encoded_df = pd.DataFrame(encoded_data_dense, columns=encoder.get_feature_names_out(['Sex']))
X = pd.concat([X.drop('Sex', axis=1), encoded_df], axis=1)

# Split dataset into training and testing sets
X_train_titanic, X_test_titanic, y_train_titanic, y_test_titanic = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling for better performance of KNN
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_titanic)
X_test_scaled = scaler.transform(X_test_titanic)

# Creating the KNN model
knn = KNeighborsClassifier(n_neighbors=7)  # You can adjust the number of neighbors

# Fitting the model with the training data
knn.fit(X_train_scaled, y_train_titanic)

y_pred_titanic = knn.predict(X_test_scaled)
accuracy = accuracy_score(y_test_titanic, y_pred_titanic)
print(f"Accuracy: {accuracy}")

# Assuming new_input is a new row of the csv
# Predicting the class for the new input
new_input = pd.DataFrame([[3, 22, 0, 0, 0, 1]], 
                         columns=['Pclass', 'Age', 'SibSp', 'Parch', 'Sex_female', 'Sex_male'], 
                         index=[0])
new_input_scaled = scaler.transform(new_input)
predicted_class = knn.predict(new_input_scaled)

print(f"The predicted class for the new input is: {predicted_class[0]}")

Accuracy: 0.8041958041958042
The predicted class for the new input is: 0


In [3]:
# Regression of the data
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import root_mean_squared_error

# load data and adjust datetime column
household_df = pd.read_csv('datasets/household_power_consumption.csv', sep=';')
# Clean data
household_df = household_df.replace('?', np.nan)
household_df = household_df.dropna().reset_index(drop=True)
# Make just one datatime column
household_df['Datetime'] = pd.to_datetime(household_df['Date'] + ' ' + household_df['Time'], format='%d/%m/%Y %H:%M:%S')
household_df = household_df.drop(['Date', 'Time'], axis=1)
# Sort by datetime
household_df = household_df.sort_values(by='Datetime')
household_df = household_df.reset_index(drop=True)
# convert datetime to unix timestamp
household_df['Datetime'] = household_df['Datetime'].astype(np.int64) // 10**9  

# split datasets
X = household_df.drop(['Global_active_power'], axis=1)  # Features
y = household_df['Global_active_power'].astype(float)  # Target variable
# Split dataset into training and testing sets based on datetime
train_percentage = 0.8
X_train = X.iloc[:int(len(X) * train_percentage)]
y_train = y.iloc[:int(len(y) * train_percentage)]
X_test = X.iloc[int(len(X) * train_percentage):]
y_test = y.iloc[int(len(y) * train_percentage):]

# Feature scaling for better performance of KNN
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Creating the KNN model for regression
knn_regressor = KNeighborsRegressor(n_neighbors=7)  # You can adjust the number of neighbors

# Fitting the model with the training data
knn_regressor.fit(X_train_scaled, y_train)

# Validate
y_pred = knn_regressor.predict(X_test_scaled)
rmse = root_mean_squared_error(y_test, y_pred)
print(f"Root Mean Squared Error: {rmse}")

/var/folders/6q/nylkt2zx4vx8clcb_x8whb100000gn/T/ipykernel_45764/2902111001.py:6: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  household_df = pd.read_csv('datasets/household_power_consumption.csv', sep=';')


Root Mean Squared Error: 0.10335349953197798


### Linear Regression with Least Squares

__Linear regression__ is a type of _regression analysis_ used for predicting the value of a _continuous dependent variable_. It works by finding the _line that best fits the data_.

_Least squares_ is a method for finding the _best-fitting_ line by __minimizing__ the _sum of the squared differences_ between the predicted and actual values.

In [4]:
from sklearn.linear_model import LinearRegression

# Creating the Linear Regression model
linear_regressor = LinearRegression()

# Fitting the model with the training data
linear_regressor.fit(X_train_scaled, y_train)

# Test the model 
y_pred_linear = linear_regressor.predict(X_test_scaled)
rmse_linear = root_mean_squared_error(y_test, y_pred_linear)
print(f"Linear Regression Root Mean Squared Error: {rmse_linear}")

Linear Regression Root Mean Squared Error: 0.03977637194254275


### Regularization with Ridge and Lasso

__Ridge regression__ (_L2_) and __Lasso regression__ (_L1_) are a type of _linear regression_ that includes a _penalty_ term to __prevent overfitting__. They work by adding a _regularization term_ to the least squares objective function

In [5]:
# implementing Rigde Regression (L2 regularization)
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# Creating the Ridge Regression model
# alpha is the regularization strength; larger values specify stronger regularization.
ridge_regressor = Ridge(alpha=1.0)

# Fitting the model with the training data
ridge_regressor.fit(X_train_scaled, y_train)

# Making predictions on the test set
y_pred = ridge_regressor.predict(X_test_scaled)

# Calculating the mean squared error of the predictions
mse = root_mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.03977639096485136


In [6]:
# implementing Lasso Regression (L1 regularization)
from sklearn.linear_model import Lasso

# Creating the Lasso Regression model
# alpha is the regularization strength; larger values specify stronger regularization.
lasso_regressor = Lasso(alpha=1.0)

# Fitting the model with the training data
lasso_regressor.fit(X_train_scaled, y_train)

# Making predictions on the test set
y_pred = lasso_regressor.predict(X_test_scaled)

# Calculating the root mean squared error of the predictions
mse = root_mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# To understand feature sensitivity, you can look at the coefficients
coefficients = pd.DataFrame(lasso_regressor.coef_, X.columns, columns=['Coefficient'])
print(coefficients)

Mean Squared Error: 0.8253175294821335
                       Coefficient
Global_reactive_power     0.000000
Voltage                  -0.000000
Global_intensity          0.091952
Sub_metering_1            0.000000
Sub_metering_2            0.000000
Sub_metering_3            0.000000
Datetime                 -0.000000


### Polynomial Regression

__Polynomial regression__ is a type of r_egression analysis_ that models
the _relationship_ between the independent and dependent variables as
an $nth-degree$ _polynomial_. It can capture _non-linear relationships_ between the variables.

In [7]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Transforming the features into polynomial features
degree = 3  # Degree of the polynomial features
poly_features = PolynomialFeatures(degree=degree)
X_train_poly = poly_features.fit_transform(X_train_scaled)
X_test_poly = poly_features.transform(X_test_scaled)

# Creating the Linear Regression model
linear_regressor = LinearRegression()

# Fitting the model with the polynomial features and the training data
linear_regressor.fit(X_train_poly, y_train_titanic)

# Making predictions on the test set
y_pred = linear_regressor.predict(X_test_poly)

# Calculating the mean squared error of the predictions
mse = root_mean_squared_error(y_test_titanic, y_pred)
print(f"Mean Squared Error: {mse}")

ValueError: Found input variables with inconsistent numbers of samples: [1639424, 571]

### Logistic Regression

__Logistic regression__ is a type of _regression analysis_ used for predicting the outcome of a _categorical dependent variable_.
It is used for __binary classification__ tasks, where the output is a
probability between $0$ and $1$.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_titanic)
X_test_scaled = scaler.transform(X_test_titanic)

# Creating the Logistic Regression model
logistic_regressor = LogisticRegression()

# Fitting the model with the training data
logistic_regressor.fit(X_train_scaled, y_train_titanic)

# Making predictions on the test set
y_pred = logistic_regressor.predict(X_test_scaled)

# Calculating the accuracy of the predictions
accuracy = accuracy_score(y_test_titanic, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7552447552447552


### Cross-Validation

__Cross-validation__ is a technique for _assessing the performance_ of a
model. It involves _splitting_ the data into multiple subsets, training the model on some subsets, and evaluating it on others.

__Cross-validation__ helps to _reduce overfitting_ and provides a more
accurate estimate of the model’s performance.

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import pandas as pd

# Create a logistic regression model
model = LogisticRegression()

# Perform cross-validation
scores = cross_val_score(model, X_train_titanic, y_train_titanic, cv=5, scoring='accuracy')

# Print the accuracy for each fold
print("Accuracy for each fold: ", scores)

# Print the mean accuracy of all 5 folds
mean_accuracy = scores.mean()
print("Mean cross-validation accuracy: ", mean_accuracy)

Accuracy for each fold:  [0.79130435 0.76315789 0.8245614  0.88596491 0.8245614 ]
Mean cross-validation accuracy:  0.8179099923722349


### Encoding

__One-hot encoding__ is a technique for _converting_ _categorical_ variables into _numerical_ variables.

It creates a _binary vector_ for each _category_, with a $1$ for the
category and $0$s for all other categories

In [9]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Creating a fake DataFrame
data = {
    'Color': ['Red', 'Blue', 'Green', 'Blue', 'Red'],
    'Size': ['S', 'M', 'L', 'XL', 'S'],
    'Price': [10, 15, 20, 25, 10]
}
df = pd.DataFrame(data)

# Display the original DataFrame
print("Original DataFrame:")
print(df)

# Applying OneHotEncoder
encoder = OneHotEncoder()
encoded_data = encoder.fit_transform(df[['Color', 'Size']]) # Encoding 'Color' and 'Size' columns
encoded_data_dense = encoded_data.toarray()

# Creating a DataFrame with the encoded data
encoded_df = pd.DataFrame(encoded_data_dense, columns=encoder.get_feature_names_out(['Color', 'Size']))

# Concatenating the encoded columns with the original DataFrame (excluding the original 'Color' and 'Size' columns)
final_df = pd.concat([df.drop(['Color', 'Size'], axis=1), encoded_df], axis=1)

# Display the final DataFrame after one-hot encoding
print("\nDataFrame after OneHotEncoding:")
print(final_df)

Original DataFrame:
   Color Size  Price
0    Red    S     10
1   Blue    M     15
2  Green    L     20
3   Blue   XL     25
4    Red    S     10

DataFrame after OneHotEncoding:
   Price  Color_Blue  Color_Green  Color_Red  Size_L  Size_M  Size_S  Size_XL
0     10         0.0          0.0        1.0     0.0     0.0     1.0      0.0
1     15         1.0          0.0        0.0     0.0     1.0     0.0      0.0
2     20         0.0          1.0        0.0     1.0     0.0     0.0      0.0
3     25         1.0          0.0        0.0     0.0     0.0     0.0      1.0
4     10         0.0          0.0        1.0     0.0     0.0     1.0      0.0


# Supervised Machine Learning Algorithms

### Random Forest

__Random forest__ is an _ensemble learning_ method that combines
_multiple decision trees_ to create a strong predictive model.

It works by building _multiple trees_ and averaging their predictions to
_reduce overfitting_.

In [10]:
def one_hot_encoder(df, column):
    """
    One-hot encodes specified columns in a DataFrame.
    
    Args:
        df (pd.DataFrame): The input DataFrame.
        column (str): Column name to be one-hot encoded.
    
    Returns:
        A DataFrame with one-hot encoded column.
    """
    encoder = OneHotEncoder()
    encoded_data = encoder.fit_transform(df[[column]])  # Encoding specified column
    encoded_data_dense = encoded_data.toarray()
    encoded_df = pd.DataFrame(encoded_data_dense, columns=encoder.get_feature_names_out([column]))
    df = pd.concat([df.drop(column, axis=1), encoded_df], axis=1)
    return df

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score

# Load the dataset
students_df = pd.read_csv('datasets/student-performance.csv')

# Preprocess the data
# Assuming the target variable is 'Grade' and it's categorical

students_df = one_hot_encoder(students_df, 'school')
students_df = one_hot_encoder(students_df, 'sex')
students_df = one_hot_encoder(students_df, 'address')
students_df = one_hot_encoder(students_df, 'famsize')
students_df = one_hot_encoder(students_df, 'Pstatus')
students_df = one_hot_encoder(students_df, 'Mjob')
students_df = one_hot_encoder(students_df, 'Fjob')
students_df = one_hot_encoder(students_df, 'reason')
students_df = one_hot_encoder(students_df, 'guardian')
students_df = one_hot_encoder(students_df, 'schoolsup')
students_df = one_hot_encoder(students_df, 'famsup')
students_df = one_hot_encoder(students_df, 'paid')
students_df = one_hot_encoder(students_df, 'activities')
students_df = one_hot_encoder(students_df, 'nursery')
students_df = one_hot_encoder(students_df, 'higher')
students_df = one_hot_encoder(students_df, 'internet')
students_df = one_hot_encoder(students_df, 'romantic')

X = students_df.drop(['G3'], axis=1)  # Features
y = students_df['G3']  # Target

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)   

# Train the model
clf = RandomForestRegressor()
# Add hyperparameters optimization
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(clf, param_grid, cv=5)

grid_search.fit(X_train, y_train)
print(f"Best parameters: {grid_search.best_params_}")
clf = grid_search.best_estimator_

clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Calculate metrics
root_mean_squared_error = root_mean_squared_error(y_test, y_pred)
print(f"Root Mean Squared Error: {root_mean_squared_error}")


Best parameters: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}
Root Mean Squared Error: 1.1677842116559312


### Gradient Boosted Decision Trees

__Gradient boosted decision trees__ are an _ensemble learning_ method
that combines _multiple decision trees_ and _gradient descedent
optimization_ to create a strong predictive model.

They work by building _trees sequentially_, with each tree _correcting the
errors_ of the previous trees.

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_titanic)
X_test_scaled = scaler.transform(X_test_titanic)

print(X_train_scaled.shape, y_train_titanic.shape)
print(X_test_scaled.shape, y_test_titanic.shape)

(571, 6) (571,)
(143, 6) (143,)


In [ ]:
#!pip install xgboost

import xgboost as xgb

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_titanic)
X_test_scaled = scaler.transform(X_test_titanic)

print(X_train_scaled.shape, y_train_titanic.shape)

model = xgb.XGBClassifier(eval_metric='mlogloss', objective='multi:softmax')
model.fit(X_train_scaled, y_train_titanic)

# Make predictions
y_pred = model.predict(X_test_scaled)

# Calculate metrics
accuracy = accuracy_score(y_test_titanic, y_pred)

print(f"Accuracy: {accuracy}")


XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/casierrav/.pyenv/versions/ud/lib/python3.13/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: '@rpath/libomp.dylib'\n  Referenced from: '/Users/casierrav/.pyenv/versions/3.13.3/envs/ud/lib/python3.13/site-packages/xgboost/lib/libxgboost.dylib'\n  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/Users/casierrav/.pyenv/versions/3.13.3/lib/libomp.dylib' (no such file), '/Users/casierrav/.pyenv/versions/3.13.3/lib/libomp.dylib' (no such file), '/Users/casierrav/.pyenv/versions/3.13.3/

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import LabelEncoder

# Load the dataset
df = pd.read_csv('datasets_kaggle/insurance.csv')

# Preprocess the data
# Assuming 'charges' is the target variable
X = df.drop('charges', axis=1)
y = df['charges']

# Encode categorical variables
for column in X.select_dtypes(include=['object']).columns:
    X[column] = LabelEncoder().fit_transform(X[column])

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Build the XGBoost model
model = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 1.0,
                max_depth = 20, alpha = 10, n_estimators = 20)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)

print(f"MSE: {mse}")

### Neural Networks

__Neural networks__ are a type of _machine learning_ model inspired by
the _human brain_.

They consist of _layers of interconnected nodes_ that process input data
and produce output data.

In [13]:
!pip install tensorflow
!pip install keras

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Load the dataset
#df = pd.read_csv('datasets_kaggle/Student_performance_data _.csv')

# Preprocess the data
# Assuming 'GradeClass' is the target variable and it's categorical
#X = df.drop('GradeClass', axis=1).values
#y = LabelEncoder().fit_transform(df['GradeClass'].values)
#y = to_categorical(y)  # Convert labels to one-hot encoding

# Normalize features
#scaler = StandardScaler()
#X = scaler.fit_transform(X)

# Split the dataset
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Build the neural network
model = Sequential()
model.add(Dense(64, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=0)

# Evaluate the model
root_mean_squared_error = root_mean_squared_error(y_test, y_pred)
print(f"Root Mean Squared Error: {root_mean_squared_error}")


ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
ERROR: No matching distribution found for tensorflow
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 100.6 kB/s eta 0:00:00 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 170.3 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 667.7/667.7 kB 548.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 643.0/643.0 kB 722.5 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


ModuleNotFoundError: No module named 'tensorflow'

# Model Evaluation

### Confusion Matrices

A __confusion matrix__ is a table that _summarizes the performance_ of a
classification model.

It shows the number of _true positives_, _true negatives_, _false positives_,
and _false negatives_.

In [14]:
from sklearn.metrics import confusion_matrix

# Calculate the confusion matrix
cm = confusion_matrix(y_test_titanic, y_pred_titanic)

print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[75 12]
 [16 40]]


### Basic Metrics

- __Accuracy:__ The proportion of correct predictions.
- __Precision:__ The proportion of true positives among all positive
predictions.
- __Recall:__ The proportion of true positives among all actual positives.
- __F1 Score:__ The harmonic mean of precision and recall.

In [15]:
# Assuming cm is the confusion matrix obtained from the Random Forest output
TN, FP, FN, TP = cm.ravel()

# Calculate metrics
accuracy = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)

# Print metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1_score}")

Accuracy: 0.8041958041958042
Precision: 0.7692307692307693
Recall: 0.7142857142857143
F1 Score: 0.7407407407407408


In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming y_true and y_pred are defined
# y_true = [...]  # True labels
# y_pred = [...]  # Predictions made by the Random Forest model

# Calculate metrics
accuracy = accuracy_score(y_test_titanic, y_pred_titanic)
precision = precision_score(y_test_titanic, y_pred_titanic, average='binary')
recall = recall_score(y_test_titanic, y_pred_titanic, average='binary')
f1 = f1_score(y_test_titanic, y_pred_titanic, average='binary')

# Print metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.8041958041958042
Precision: 0.7692307692307693
Recall: 0.7142857142857143
F1 Score: 0.7407407407407407


### Classifier Decision Metrics

- __ROC Curve:__ A plot of the true positive rate against the false positive rate.
- __Precision-Recall Curve:__ A plot of precision against recall.
- __AUC-ROC:__ The area under the ROC curve.
- __AUC-PR:__ The area under the precision-recall curve.

In [18]:
!pip install matplotlib
from sklearn.metrics import roc_curve, precision_recall_curve, auc
import matplotlib.pyplot as plt

# Assuming y_true and y_pred_proba are defined
# y_true = [...]  # True labels
# y_pred_proba = [...]  # Predicted probabilities for the positive class

# Calculate ROC Curve
fpr, tpr, thresholds_roc = roc_curve(y_test_titanic, y_pred_titanic)
roc_auc = auc(fpr, tpr)

# Plot ROC Curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

# Calculate Precision-Recall Curve
precision, recall, thresholds_pr = precision_recall_curve(y_test_titanic, y_pred_titanic)
pr_auc = auc(recall, precision)

# Plot Precision-Recall Curve
plt.figure()
plt.plot(recall, precision, color='blue', lw=2, label=f'Precision-Recall curve (area = {pr_auc:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")
plt.show()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 40.3 kB/s eta 0:00:0000:0200:09m
^C
Traceback (most recent call last):
  File "/Users/casierrav/.pyenv/versions/ud/bin/pip", line 8, in <module>
    sys.exit(main())
             ~~~~^^
  File "/Users/casierrav/.pyenv/versions/3.13.3/envs/ud/lib/python3.13/site-packages/pip/_internal/cli/main.py", line 80, in main
    return command.main(cmd_args)
           ~~~~~~~~~~~~^^^^^^^^^^
  File "/Users/casierrav/.pyenv/versions/3.13.3/envs/ud/lib/python3.13/site-packages/pip/_internal/cli/base_command.py", line 157, in main
    with self.main_context():
         ~~~~~~~~~~~~~~~~~^^
  File "/Users/casierrav/.pyenv/versions/3.13.3/lib/python3.13/contextlib.py", line 148, in __exit__
    next(self.gen)
    ~~~~^^^^^^^^^^
  File "/Users/casierrav/.pyenv/versions/3.13.3/envs/ud/lib/python3.13/site-packages/pip/_internal/cli/command_context.py", line 19, in main_context
    with self._main_context:
         ^^^^^^^^^^^^^^^^^^
  File "/Users/cas

ModuleNotFoundError: No module named 'matplotlib'

### Regression Evaluation Metrics

- __Mean Squared Error:__ The average of the squared differences between the predicted and actual values.
- __Mean Absolute Error:__ The average of the absolute differences between the predicted and actual values.
- __R-Squared:__ The proportion of the variance in the dependent variable that is predictable from the independent variables.
- __Adjusted R-Squared:__ A modified version of R-squared that adjusts for the number of predictors in the model.
- __Root Mean Squared Error:__ The square root of the mean squared error.

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Assuming y_true and y_pred are defined
# y_true = [...]  # True values
# y_pred = [...]  # Predictions made by the XGBoost regressor

# Number of observations and predictors
n = len(y_true)  # Number of observations
p = model.n_features_in_

# Calculate metrics
mse = mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)
r_squared = r2_score(y_true, y_pred)
adjusted_r_squared = 1 - (1-r_squared) * (n-1) / (n-p-1)
rmse = np.sqrt(mse)

# Print metrics
print(f"MSE: {mse}")
print(f"MAE: {mae}")
print(f"R-squared: {r_squared}")
print(f"Adjusted R-squared: {adjusted_r_squared}")
print(f"RMSE: {rmse}")